In [ ]:
import numpy as np
import pandas as pd
import math
from concurrent.futures import ThreadPoolExecutor
from sympy import symbols
from sympy.solvers.solveset import nonlinsolve
import time
import logging

number_of_sim = 180
number_of_drive = 1
numberofsmallelements = 2000

# 定义常数和材料属性
E = 169e9
rho = 2330
t = 25e-6
eps_0 = 8.85e-12
beta = 4.730041
alpha_n = (np.sin(beta) - np.sinh(beta)) / (np.cosh(beta) - np.cos(beta))
pi_160 =(160 / 180) * math.pi
pi_10 = (10 / 180) * math.pi



# 定义变量范围
w_t_values = np.arange(5e-6, 20e-6, 1e-6) #16
l_t_values = np.arange(100e-6, 1000e-6, 50e-6) #19
Q_values = np.arange(5000, 50000 , 2500) # 19
V_values = np.arange(1, 10, 1) # 10
d_values = np.arange(2e-6, 10e-6, 1e-6) #9


# electrode_length_values = np.arange(100e-6, 900e-6 , 10e-6) #81
# electrode_width_values = np.arange(5e-6, 30e-6, 1e-6) #26
# w_c_values = np.arange(5e-6, 20e-6, 0.2e-6) #76 
# l_c_values = np.arange(5e-6, 30e-6 , 1e-6) #26



# w_t_values = [8e-6]
# l_t_values = [800e-6]
electrode_length = [700e-6]
electrode_width = [20e-6]
w_c= [10e-6]
l_c = [20e-6]
# Q_values = [20000]
V_values = [6.4]
d_values = [2e-6]



In [ ]:
def calculate_for_parameters(w_t, l_t, electrode_length, electrode_width, w_c, l_c, V, d, Q, E, t, rho, beta, alpha_n, numberofsmallelements,index):
    start_time = time.time()
    results = []

    length_x = np.zeros(numberofsmallelements)
    modeshape_unnormalized = np.zeros(numberofsmallelements)
    second_derivative = np.zeros(numberofsmallelements)
    first_derivative = np.zeros(numberofsmallelements)
    
    length_x = np.linspace(0, l_t- l_t/2000, 2000)
    modeshape_unnormalized = -(-np.sin(beta * length_x / l_t) + np.sinh(beta * length_x / l_t) +
                                alpha_n * (-np.cos(beta * length_x / l_t) + np.cosh(beta * length_x / l_t)))
    second_derivative = -1 / np.max(modeshape_unnormalized) * (beta ** 2) * (np.sin(beta * length_x / l_t) + np.sinh(beta * length_x / l_t) + alpha_n * (np.cos(beta * length_x / l_t) + np.cosh(beta * length_x / l_t)))
    first_derivative = -1 / np.max(modeshape_unnormalized) * beta * (-np.cos(beta * length_x / l_t) + np.sinh(beta * length_x / l_t) + alpha_n * (np.sin(beta * length_x / l_t) + np.cosh(beta * length_x / l_t)))

    modeshape1 = modeshape_unnormalized / np.max(modeshape_unnormalized)
    
    dx = length_x[1] - length_x[0]
    m_coef_b = np.sum(modeshape1 ** 2 * dx / l_t)
    k_coef_b = np.sum(second_derivative ** 2 * dx / l_t) 
    k_coef_b3 = np.sum(first_derivative ** 2 * dx / l_t)

    k_tt = k_coef_b / 12 * E * t * ((w_t / l_t) ** 3)
    k_t3 = k_coef_b3 * E * t * w_t / (l_t ** 3)
    k_t = k_tt
    Mass = rho * (t * w_t * l_t * m_coef_b + electrode_length * electrode_width * t + w_c * l_c * t *2)
    omega_0 = math.sqrt(k_t / Mass)
    freq_0 = omega_0 / (2 * math.pi)
    eps_0 = 8.85e-12
    trans_factor = eps_0 * V * electrode_length * t / (d ** 2)
    k_e = 2 * trans_factor * V / d
    k_e3 = 4 * trans_factor * V / (d ** 3)      
    c = math.sqrt(Mass * k_t) / Q
    
    number_of_drive = 1
    number_of_sim = 180
    
    freq = np.zeros((number_of_sim, number_of_drive))
    m_c = np.zeros((number_of_sim, number_of_drive))
    
    for ii in range(number_of_drive):
        vac = 5e-3 + 0.1e-3 * ii        
        force_ac = vac * trans_factor
        vac_round = np.round(vac / 1e-3 , 2)
        for i in range(number_of_sim):
            x, y = symbols('x, y', reals=True)
            solutions = nonlinsolve([-Mass * (x ** 2) * y + (k_t - k_e) * y + (k_t3 - k_e3) * (y ** 3) * 3 / 4 - force_ac * math.cos(0), 
                                c * x * y - force_ac * math.sin(0)], [x, y])
            for j in range(len(solutions)):
                sol = solutions.args[j]
                if np.abs(sol[0]) > omega_0 / 2 and np.abs(sol[0]) < omega_0 * 3 / 2 and sol[0] > 0:
                    freq[i, ii] = sol[0] / (2 * math.pi)
                    m_c[i, ii] = sol[1] * freq[i, ii] * 2 * math.pi * trans_factor / 1e-9

    results.append({
        'w_t': w_t, 'l_t': l_t, 'electrode_length': electrode_length, 'electrode_width': electrode_width, 'w_c': w_c,
        'l_c': l_c, 'Q': Q, 'V': V, 'd': d, 'Vac_ground': vac, 'freq': freq, 'm_c': m_c
    })
    end_time = time.time()
    print(f"Thread {index}: End - Duration: {end_time - start_time:.2f} seconds")
    return results

# 多线程执行
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    index =0
    for l_t in l_t_values:
        for w_t in w_t_values:
            for V in V_values:
                for Q in Q_values:
                    for d in d_values:
                        index += 1
                        future = executor.submit(calculate_for_parameters, w_t, l_t, electrode_length, electrode_width, w_c, l_c, V, d, Q, E, t, rho, beta, alpha_n, numberofsmallelements,index)
                        futures.append(future)
    
    results = [f.result() for f in futures]

# 组合结果并保存
final_results = [item for sublist in results for item in sublist]
results_df = pd.DataFrame(final_results)
results_df.to_hdf('filename.h5', 'mydata', mode='w')